# Running Lammps simulations with Pymatgen

Notice the easy lammps-cython interface for adding pymatgen structures.

Here I wanted to investigate if there is any difference between `buck/coul/long` and `buck` + `coul/long` and no there is no difference.

In [1]:
import pymatgen as pmg
from pymatgen.io.cif import CifParser
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer

import lammps

In [2]:
structure = CifParser('../data/structure/MgO.cif').get_structures()[0]
structure.remove_oxidation_states()
spga = SpacegroupAnalyzer(structure)
conventional_structure = spga.get_conventional_standard_structure() * (5, 5, 5)

In [3]:
print('num atoms', len(conventional_structure))

num atoms 1000


In [4]:
elements = pmg.Element('Mg'), pmg.Element('O') # specify index for each element
mg, o = elements

buckingham_potential = {
    (mg, mg): [1309362.2766468062, 0.104, 0.0],
    (mg, o): [9892.357, 0.20199, 0.0],
    (o, o): [2145.7345, 0.3, 30.2222]
}

charges = {
    mg: 1.4,
    o: -1.4
}

In [5]:
lmp = lammps.Lammps(units='metal', style='full')
elements = lmp.system.add_pymatgen_structure(conventional_structure, elements)

lmp.command('kspace_style pppm 1e-6')
for c in charges:
    lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))

lmp.command('pair_style buck/coul/long 10.0 10.0')
for p in buckingham_potential:
    lmp.command('pair_coeff %d %d %f %f %f' % (elements.index(p[0])+1, elements.index(p[1])+1, *buckingham_potential[p]))

In [6]:
%%timeit

lmp.run(0)

49.2 ms ± 859 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
print('potential energy', lmp.thermo.potential_energy.scalar)
print('temperature', lmp.thermo.temperature.scalar)
print('pressure', lmp.thermo.pressure.vector)

potential energy 1022.4233073482385
temperature 0.0
pressure [ 6.43097404e+05  6.43097404e+05  6.41907177e+05  5.50050186e-10
 -1.87660923e-09 -1.99158064e-09]


In [8]:
lmp = lammps.Lammps(units='metal', style='full')
elements = lmp.system.add_pymatgen_structure(conventional_structure, elements)

lmp.command('kspace_style pppm 1e-6')
for c in charges:
    lmp.command('set atom %d charge %f' % (elements.index(c)+1, charges[c]))

lmp.command('pair_style hybrid/overlay coul/long 10.0 buck 10.0')
lmp.command('pair_coeff * * coul/long')
for p in buckingham_potential:
    lmp.command('pair_coeff %d %d buck %f %f %f' % (elements.index(p[0])+1, elements.index(p[1])+1, *buckingham_potential[p]))    

In [9]:
%%timeit

lmp.run(0)

57.7 ms ± 393 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
print('potential energy', lmp.thermo.potential_energy.scalar)
print('temperature', lmp.thermo.temperature.scalar)
print('pressure', lmp.thermo.pressure.vector)

potential energy 1022.4233073482385
temperature 0.0
pressure [ 6.43097404e+05  6.43097404e+05  6.41907177e+05  5.50050186e-10
 -1.87660923e-09 -1.99158064e-09]
